# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix
import datetime

import warnings
warnings.filterwarnings('ignore')

### 2. Load the dataset

In [ ]:
df= pd.read_csv('Dataset/Car Data.csv')

In [ ]:
df.head(5)

### 3. Check the shape and basic information of the dataset.

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [ ]:
df.duplicated().sum()

### 5. Drop the columns which you think redundant for the analysis.

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df.drop('Car_Name', inplace=True, axis=1)

In [ ]:
df.info()

In [ ]:
df.shape

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [ ]:
df['age_of_the_car']=datetime.datetime.today().year-df['Year']

In [ ]:
# After adding 'age_of_the_car' column and before dropping the year column
df.head(5)

In [ ]:
df.drop('Year', axis=1,inplace=True)

In [ ]:
# After dropping the year column
df.head(5)

### 7. Encode the categorical columns

In [ ]:
cate = ['Fuel_Type','Seller_Type','Transmission',]
lbl_encode = LabelEncoder()

for i in cate:
    df[i]=df[[i]].apply(lbl_encode.fit_transform)
df.head(5)    

### 8. Separate the target and independent features.

In [ ]:
X = df.drop('Selling_Price',axis=1)
y = df['Selling_Price']
df.head(5)

### 9. Split the data into train and test.

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

In [ ]:
## Let us check the accuracy to see hows our model is performing
# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

rf = RandomForestRegressor()
rf.fit(X_train,y_train)

### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

r2_train = r2_score(y_train,y_train_pred)
r2_test = r2_score(y_test,y_test_pred)

print('r2-score train:',r2_train)
print('r2-score test',r2_test)

In [ ]:
plt.figure(figsize=(10,10))
sns.regplot(y_test, y_test_pred, fit_reg=True, scatter_kws={"s": 100})

### 11. Create a pickle file with an extension as .pkl

In [ ]:
import pickle
# Saving model to disk
pickle.dump(rf, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>

<body>

    <div class="hero-image">
      <div class="hero-text">

        <h1 style="font-size:50px">Used Car Price Predictor</h1>
         <br><br><h3>{{ prediction_text }}<h3>
      </div>
    </div>

     <style>

        body, html {
          height: 100%;
          margin: 0;
          font-family: Arial, Helvetica, sans-serif;
        }

        .hero-image {
          background-image: linear-gradient(rgba(0, 0, 0, 0.5), rgba(0, 0, 0, 0.5)), url('/static/image.jpg');
          height: 50%;
          background-position: bottom;
          background-repeat: no-repeat;
          background-size: cover;
          position: relative;
        }

        .hero-text {
          text-align: center;
          position: absolute;
          top: 50%;
          left: 50%;
          transform: translate(-50%, -50%);
          color: white;
        }

    </style>


    <div style="color:	rgb(0, 0, 0)">
        <form action="{{ url_for('predict')}}" method="post">
            <h2>Enter Car Details: </h2>
            <h3>Age of the car(In years)</h3>
            <input id="first" name="Age_of_the_car" type="number ">
            <h3>Present Showroom Price(In lakhs)</h3><br><input id="second" name="Present_Price" required="required">
            <h3>Kilometers Driven</h3><input id="third" name="Kms_Driven" required="required">
            <h3>Owner Type (0/1/3)</h3><br><input id="fourth" name="Owner" required="required">
            <h3>Fuel type</h3><br><select name="Fuel_Type" id="fuel" required="required">
                <option value="0">Petrol</option>
                <option value="1">Diesel</option>
                <option value="2">CNG</option>
            </select>
            <h3>Seller Type</h3><br><select name="Seller_Type" id="resea" required="required">
                <option value="0">Dealer</option>
                <option value="1">Individual</option>
            </select>
            <h3>Transmission type</h3><br><select name="Transmission" id="research" required="required">
                <option value="0">Manual Car</option>
                <option value="1">Automatic Car</option>
            </select>
            <br><br><button id="sub" type="submit ">Predict Selling Price</button>
            <br>


        </form>

    </div>

    <style>
	body {
            background-color: 101, 10, 20;
            text-align: center;
            padding: 0px;
	    font-family: Helvetica;
        }

        #research {
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;
        }

        #box {
            border-radius: 60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }

        #fuel {
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }

        #fuel:hover {
            background-color: white;
        }

        #research {
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #research:hover {
            background-color: white;
        }

        #resea {
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #resea:hover {
            background-color: white;
        }

        #sub {
            background-color: Green;
            font-family:'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align: center;
            border-radius: 20px;
            font-size: 18px;
            color: white;
        }

        #sub:hover {
            background-color: white;
        }

        #first {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #second {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #third {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #fourth {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
    </style>
</body>

</html>

### b) Create app.py file and write the predict function

In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)  # Initialize the flask App
model = pickle.load(open('model.pkl', 'rb'))  # loading the trained model


@app.route('/')  # Homepage
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    if request.method == 'POST':
        Present_Price = float(request.form['Present_Price'])
        Kms_Driven = int(request.form['Kms_Driven'])
        Owner = int(request.form['Owner'])
        Fuel_Type = int(request.form['Fuel_Type'])
        Age_of_the_car = int(request.form['Age_of_the_car'])
        Seller_Type = int(request.form['Seller_Type'])
        Transmission = int(request.form['Transmission'])
        prediction = model.predict(
            [[Present_Price, Kms_Driven, Owner, Fuel_Type, Age_of_the_car, Seller_Type, Transmission]])
        output = round(prediction[0], 2)
        # rendering the predicted result
        return render_template('index.html', prediction_text="You can sell your car at {} lakhs".format(output))


if __name__ == "__main__":
    app.run(debug=True)

### 13. Deploy your app on Heroku. (write commands for deployment)

## -- Heroku Signup not working, will update once available --

#### Above we performed the following steps:
- Created a newfolder/new project in vscode/pycharm.
- Created virtual environment and installed the requirement packages.
- Created templates, static, app.py (once the app is running in your local machine, lets proceed with deployment)

#### Follow the below steps to deploy your application on heroku
- First We need login to heroku and create a heroku app.
    - heroku login (then login to heroku with your the id and password).
    
- Create a heroku app

    - heroku create nameofyourapp (go back to heroku dashboard we can see a app created with given name)

- Create a Procfile (Procfile is a mechanism for declaring what commands are run by your application’s dynos on the Heroku platform.)

    - echo web: >Procfile (creates a procfile)
    - web: gunicorn app:app(paste this in the procfile)

- Create a runtime.txt file (need to tell what type of python need to be used)

    - python-3.10.5 (pass your version of the python in the file)

- Create a requirements.txt file

    - pip freeze >requirements.txt

- initialize a git repository git init

- Push your code to git repository

    - git add .
    - git commit -m 'Intial Commit'

- Push the code from your local repository's main branch to your heroku remote.

    - heroku git:remote -a nameofyourapp (this will set git remote heroku)
    - git push heroku master

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

## -- Heroku Signup not working, will update once available --

In [ ]:
##Final URL
https://sidglcarapp.herokuapp.com/

### Happy Learning :)